<a href="https://colab.research.google.com/github/MocT117/Another-one-/blob/master/05_KPI_Import_Process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import config

st.logo(
    image=config.horizontal,
    size = "large",
    icon_image=config.icon
    )

st.set_page_config(page_title="Semáforo Pago Pedimento", page_icon=":bar_chart:", layout="centered")

st.title("KPI - Import Process")

if "df_base" in st.session_state:
    df = st.session_state["df_base"]
else:
    st.warning("Primero ve a 'Supply Chain Process - Order Management' para cargar el archivo.")

# Convertir columnas a fecha
df['Fecha entrada Aduana'] = pd.to_datetime(df['Fecha entrada Aduana'], errors='coerce')
df['Fecha pago pedimento'] = pd.to_datetime(df['Fecha pago pedimento'], errors='coerce')

# Filtrar filas válidas (con ambas fechas)
df_validas = df[df['Fecha entrada Aduana'].notna() & df['Fecha pago pedimento'].notna()].copy()

# Calcular diferencia en días hábiles
df_validas['Diferencia días hábiles'] = df_validas.apply(
    lambda x: np.busday_count(
        x['Fecha entrada Aduana'].date(),
        x['Fecha pago pedimento'].date()
    ) if pd.notna(x['Fecha entrada Aduana']) and pd.notna(x['Fecha pago pedimento']) else np.nan,
    axis=1
)

# Semáforo: On time <= 7 días hábiles
def status_semáforo(dias):
    if pd.isna(dias):
        return 'Sin fecha'
    elif dias <= 7:
        return 'On time'
    elif dias > 7:
        return 'Delay'
    else:
        return 'Revisar'
df_validas['Semáforo'] = df_validas['Diferencia días hábiles'].apply(status_semáforo)

num_filas = len(df_validas)

df_validas['mes'] = df_validas['Fecha de PO'].dt.to_period('M')

meses_opciones = ["Todos"] + [str(m) for m in sorted(df_validas['mes'].dropna().unique())]
mes_elegido = st.selectbox("📅 Selecciona un mes:", options=meses_opciones)

if mes_elegido != "Todos":
    df_filtrado = df_validas[df_validas['mes'].astype(str) == mes_elegido]
else:
    df_filtrado = df_validas

# Mostrar tabla
st.subheader("Fecha entrada Aduana vs Fecha pago pedimento")
st.dataframe(df_filtrado[['PO', 'Fecha entrada Aduana', 'Fecha pago pedimento', 'Diferencia días hábiles', 'Semáforo']])

# Gauge chart: promedio días hábiles
st.subheader("Semáforo Fecha entrada Aduana vs Fecha pago pedimento (días hábiles)")

if not df_filtrado['Diferencia días hábiles'].dropna().empty:
    promedio_dias = df_filtrado['Diferencia días hábiles'].mean().round(0)
else:
    promedio_dias = 0

minimo = 0
maximo = max(14, int((df_filtrado['Diferencia días hábiles'].max() // 4 + 1) * 4))
tick_vals = list(range(minimo, maximo+1, 3))

fig = go.Figure(go.Indicator(
    mode = "gauge+number+delta",
    delta={'reference': 7, 'increasing': {'color': "red"}, 'decreasing': {'color': "green"}},
    value = promedio_dias,
    number = {'suffix': " días hábiles", 'font': {'size': 36}},
    title = {'text': "Fecha entrada Aduana vs Fecha pago pedimento"},
    gauge = {
        'axis': {'range': [minimo, maximo], 'tickmode': 'array', 'tickvals': tick_vals, 'tickfont': {'size': 16}},
        'steps': [
            {'range': [minimo, 7], 'color': 'green'},
            {'range': [7, 10], 'color': 'yellow'},
            {'range': [10, 13], 'color': 'orange'},
            {'range': [13, maximo], 'color': 'red'}     # Rojo: Delay
        ],
        'bar': {'color': "black", 'thickness': 0.25}
    }
))

st.plotly_chart(fig)
st.markdown(
    f"<div style='text-align:center; font-size:18px; margin-top:-30px; margin-bottom:10px;'><b>Promedio considerando {num_filas} diferencias</b></div>",
    unsafe_allow_html=True
)

# Percentiles como texto
serie = df_validas['Diferencia días hábiles'].dropna()
percentil_50 = serie.quantile(0.5)
percentil_75 = serie.quantile(0.75)
percentil_90 = serie.quantile(0.9)
percentil_95 = serie.quantile(0.95)

st.subheader("Resumen de percentiles")
st.write(f"Percentil 50 (Mediana): **{percentil_50:.2f}** días hábiles")
st.write(f"Percentil 75: **{percentil_75:.2f}** días hábiles")
st.write(f"Percentil 90: **{percentil_90:.2f}** días hábiles")
st.write(f"Percentil 95: **{percentil_95:.2f}** días hábiles")

# ==== CURVA DE PERCENTILES (1-99) ====
st.subheader("Curva de percentiles de las diferencias (días hábiles)")

# Serie de diferencias (sin nan)
serie_percentil = df_validas['Diferencia días hábiles'].dropna().sort_values().reset_index(drop=True)

# Percentiles a graficar (del 1 al 99)
percentiles = np.arange(1, 100)
valores_percentiles = [serie_percentil.quantile(p/100) for p in percentiles]

# Graficar la curva
fig_percentiles = go.Figure()
fig_percentiles.add_trace(go.Scatter(
    x=percentiles,
    y=valores_percentiles,
    mode='lines+markers',
    line=dict(color='#27AE60', width=3),
    marker=dict(size=6),
    name='Curva percentiles'
))

# Resalta P50, P75, P90
for px, clr, label in zip([50, 75, 90], ['#F4D03F', '#F39C12', '#C0392B'], ['P50', 'P75', 'P90']):
    val = serie_percentil.quantile(px/100)
    fig_percentiles.add_trace(go.Scatter(
        x=[px], y=[val],
        mode='markers+text',
        marker=dict(size=12, color=clr, symbol='diamond'),
        text=[label],
        textposition='top center',
        name=label
    ))

fig_percentiles.update_layout(
    title="Curva de percentiles de las diferencias (días hábiles)",
    xaxis_title="Percentil (%)",
    yaxis_title="Diferencia (días hábiles)",
    plot_bgcolor="white",
    font=dict(size=15),
    height=450
)

st.plotly_chart(fig_percentiles)
